# SIGN LANGUAGE MULTILABEL CLASSIFICATION WITH FASTAI

## INSTALL AND IMPORT ALL THE NECESSARY LIBRARIES

In [ ]:
# These are all the necessary libraries we have to install 
!pip install fastai==2.2.0
!pip install -U albumentations
!pip install opencv-python==4.5.4.60
!pip install -q iterative-stratification

In [ ]:
#import albumentations for data augmentation
import albumentations
#fastai is built on top of torch so import it
import torch

torch.__version__

In [ ]:
#import everything from fastai
from fastai.vision.all import *
#import multilabelstratifiedkfold to be used to create folds
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

## DATA LOADING AND VIEWING 

In [ ]:
# loading the train data and appending the .jpg extension for easy manipulation
path = Path('../input/signprediction/sign_prediction')

train_df = pd.read_csv(path/'Train.csv')

train_df['img_IDS'] = train_df['img_IDS'].apply(str) + ".jpg"

train_df['img_IDS'] = "../input/signprediction/sign_prediction/Images/" + train_df['img_IDS']

train_df.head()

### CREATING STRATIFIED KFOLDS

In [ ]:
#creating 3 shuffled stratified kfold
strat_kfold = MultilabelStratifiedKFold(n_splits=3, random_state=42, shuffle=True)
train_df['fold'] = -1
for i, (_, test_index) in enumerate(strat_kfold.split(train_df.img_IDS.values, train_df.iloc[:,1:].values)):
    train_df.iloc[test_index, -1] = i
train_df.head()

In [ ]:
#plot the folds to ensure that they are of equal portions
train_df.fold.value_counts().plot.bar();

### CREATING A SPECIAL TRANSFORM CLASS TO USE ALBUMENTATIONS

In [ ]:
class AlbumentationsTransform (RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
#define all the augmentations you need
def get_train_aug(): return albumentations.Compose([
            albumentations.Resize(512, 512),
            # albumentations.RandomBrightness(),
            # albumentations.RandomRotate90(),
            # albumentations.Rotate(limit=(-90, 90)),
            # albumentations.Transpose(),
            # albumentations.RandomContrast(),
            # albumentations.RandomBrightnessContrast(),
            # albumentations.RandomGamma(),
            # albumentations.Blur(),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            ], p=1.)



def get_valid_aug(): return albumentations.Compose([
            albumentations.Resize(512, 512),
            ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]


### CREATING THE FASTAI DATABLOCK

In [ ]:
# to learn more about datablocks in fast you have to visit fastai.docs
def get_data(fold=0, size=224,bs=32):
    return DataBlock(blocks=(ImageBlock,MultiCategoryBlock),
                 get_x=ColReader(0),
                 get_y=ColReader(1, label_delim=' '),
                 splitter=IndexSplitter(train_df[train_df.fold == fold].index),
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms).dataloaders(train_df, bs=bs)

### CREATING YOUR OWN SPECIAL METRICS

In [ ]:
# This is not necessary since fastai already has an inbuilt accuracy_multi metrics for multilabel classifications
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    "Compute accuracy when `inp` and `targ` are the same size."
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()

In [ ]:
def F_score(output, label, threshold=0.2, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

### LOADING THE TEST DATA AND MAKE NECESSARY MODIFICATIONS

In [ ]:
#load the test data and add the .jpg extension plus the path 
test_df = pd.read_csv('../input/signprediction/sign_prediction/SampleSubmission.csv')
tstpng = test_df.copy()
tstpng['img_IDS'] = tstpng['img_IDS'].apply(str) + ".jpg"
tstpng['img_IDS'] = "../input/signprediction/sign_prediction/Images/" + tstpng['img_IDS']
tstpng.head()

## MODELLING

In [ ]:
# I am going to use mixup in my training, for more information about how mix up work go to the official fastai.docs
mixup = MixUp(0.3)

In [ ]:
import gc

In [ ]:
#in here we are going to do both our training and inference using resnet34 architecture
#I have also used alot of advanced techniques like callbacks and if you dont understand please visit the official fastai.docs
all_preds = []

for i in range(3):
    dls = get_data(i,256,64)
    learn = cnn_learner(dls, resnet34, metrics=[partial(accuracy_multi, thresh=0.2),partial(F_score, threshold=0.2)],cbs=mixup).to_fp16()
    learn.fit_one_cycle(10, cbs=EarlyStoppingCallback(monitor='valid_loss'))
    learn.dls = get_data(i,512,32)
    learn.fine_tune(10,cbs=EarlyStoppingCallback(monitor='valid_loss'))
    tst_dl = learn.dls.test_dl(tstpng)
    preds, _ = learn.get_preds(dl=tst_dl)
    all_preds.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()


## PREPARING THE SUBMISSION

In [ ]:
preds = np.mean(np.stack(all_preds), axis=0)

In [ ]:
k=[]
for col in tstpng.columns: 
  k.append(col) # creating list of the label

k

In [ ]:
test_df = pd.read_csv("../input/signprediction/sign_prediction/Test.csv")

In [ ]:
import os
submission = pd.DataFrame()
submission["ID"] = test_df["img_IDS"]
for i, c in enumerate(dls.vocab):
  print(c)
  submission[c] = preds[:,i]
submission.head()

In [ ]:
submission.to_csv('baseline_model_16.csv', index=False)